In [ ]:
# 시그모이드(Sigmoid)와 하이퍼볼릭 탄젠트(Hyperbolic Tangent)는 대표적인 활성화 함수(activation Function)이다.
# 시그모이드(sigmoid)는 음의 무한대에 가까울수록 0에 근접하는 값을 가지며, 양의 무한대에 가까울수록 1에 근접하는 값을 갖는다.
# 즉, 시그모이드 함수의 출력값의 범위는 0에서 1사이로 정해져 있다.
# 하이퍼볼릭 탄젠트(탄에이치)는 음의 무한대에 가까울수록 -1에 근접하는 값을 가지며, 양의 무한대에 가까울수록 1에 근접하는 값을 갖는다.
# 즉, 탄에이치 함수의 출력값의 범위는 -1에서 1사이이다.
# 두 함수 모두 양 극단의 기울기는 0에 근접하는 것을 특징으로 한다.

시그모이드 수식
$$
\sigma(x)=\frac{1}{1+e^{-x}}
$$

하이퍼볼릭 탄젠트 수식
$$
\text{tanh}(x)=\frac{e^x-e^{-x}}{e^x+e^{-x}}
$$

In [ ]:
# 선형 회귀의 경우, 키와 몸무게 같은 선형 데이터의 관계를 학습하는 문제이다.
# 즉, n차원의 실수 벡터를 입력으로 받아 선형 관계의 m차원의 실수 벡터를 반환하도록 학습하는 문제이다.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()
print(cancer.DESCR)
# 위스콘신 유방암 데이터셋은 30개의 속성을 가지며, 이를 통해 유방암 여부를 예측한다.

In [ ]:
# 판다스로 데이터를 변환한다.
df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
df["class"] = cancer.target

In [ ]:
sns.pairplot(df[["class"] + list(df.columns[:10])])
plt.show()

In [ ]:
# 맨 윗 줄의 경우 각 속성별 샘플의 점들이 0번 클래스에 해당하는 경우 아래쪽에, 1번 클래스에 해당하는 경우 위쪽에 찍혀 있는 것을 볼 수 있다.
# 이 점들의 클래스별 그룹이 특정 값을 기준으로 명확히 나눠진다면 좋다는 것을 확인할 수 있다.
# 마찬가지로 다른 속성들에 대해서도 페어 플롯을 그린다.

sns.pairplot(df[["class"] + list(df.columns[10:20])])
plt.show()



In [ ]:
sns.pairplot(df[["class"] + list(df.columns[20:30])])
plt.show()

In [ ]:
# 위의 그림들을 바탕으로 몇 개를 골라서 좀 더 예쁘게 표현한다.

cols = ["mean radius", "mean texture",
        "mean smoothness", "mean compactness", "mean concave points",
        "worst radius", "worst texture",
        "worst smoothness", "worst compactness", "worst concave points",
        "class"]

In [ ]:
for c in cols[:-1]:
    sns.histplot(df, x=c, hue=cols[-1], bins=50, stat='probability')
    plt.show()

In [ ]:
# 클래스에 따른 속성별 분포를 나타낸 것이다.
# 0번 클래스에는 파란색, 1번 클래스는 주황색으로 표시되어 있다.
# 겹치는 영역이 적을수록 좋은 속성이라고 판단할 수 있다.
# 완벽하게 두 클래스를 나눠주는 속성은 없지만, 부족한 부분은 다른 속성으로 보완할 수 있을 것이라 생각하고, 해당 속성들을 가지고 로지스틱 회귀를 진행한다.

In [ ]:
# 학습 코드 구현
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

data = torch.from_numpy(df[cols].values).float()

data.shape
# Split x and y.
x = data[:, :-1]
y = data[:, -1:]

print(x.shape, y.shape)

In [ ]:
# 선형 회귀와 동일한 방식으로 텐서 x와 텐서 y를 가져온다.
# 학습에 필요한 설정값을 정해준다.

# Define configurations
n_epochs = 200000
learning_rate = 1e-2
print_interval=10000

In [ ]:
# 이제 모델을 준비한다.
# 선형 회귀에서는 선형 계층 하나만 필요했지만, 이번엔 시그모이드 함수도 모델에 포함한다.
# 즉, nn.Module을 상속받아 클래스를 정의하고 내부에 필요한 계층들을 소유하도록 한다.

# nn.Module을 상속받은 자식 클래스를 정의할 때는 보통 두 개의 함수(메서드)를 오버라이드 한다.
# 또한, __init__ 함수를 통해 모델을 구성하는 데 필요한 내부 모듈을 미리 선언해 둔다.
# forward 함수는 미리 선언된 내부 모듈을 활용하여 계산을 수행한다.

# Define costum model
class MyModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        self.input_dim = input_dim
        self.output_dim = output_dim

        super().__init__()

        self.linear = nn.Linear(input_dim, output_dim)
        self.act = nn.Sigmoid()
    
    def forward(self, x):
        # |x| = (batch_size, input_dim)
        y = self.act(self.linear(x))
        # |y| = (batch_size, output_dim)
        return y

In [ ]:
# 이제 정의한 나만의 로지스특 회귀 모델 클래스를 생성하고 BCE 손실 함수와 옵티마이저도 준비한다.
# 선형 회귀와 마찬가지로 모델의 입력 크기는 텐서 x의 마지막 차원의 크기가 되고
# 출력 크기는 텐서 y의 마지막 차원 크기가 된다.

model = MyModel(input_dim=x.size(-1), output_dim=y.size(-1))
crit = nn.BCELoss() # Define BCELoss instead of MSELoss
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
# 학습을 위한 준비는 끝
# 선형 회귀와 똑같은 코드로 학습을 진행시킨다.
for i in range(n_epochs):
    y_hat = model(x)
    loss = crit(y_hat, y)
    
    optimizer.zero_grad()
    loss.backward()
    
    optimizer.step()

    if(i + 1) % print_interval == 0:
        print("Epoch %d : loss = %.4e" % (i + 1, loss))

In [ ]:
# 결과 확인
# 지금은 분류 문제이므로 예측 결과에 대한 정확도 평가가 가능하다.
# 마지막 모델을 통과하여 얻은 y_hat과 y를 비교하여 정확도를 계산한다.
correct_cnt = (y == (y_hat > .5)).sum()
total_cnt = float(y.size(0))

print("Accuracy: %.4f" % (correct_cnt / total_cnt)) # Accuracy: 0.9649

In [ ]:
# 예측된 결괏값의 분포도 살펴볼 수 있다.
df = pd.DataFrame(torch.cat([y, y_hat], dim=1).detach().numpy(), columns=["y", "y_hat"])

sns.histplot(df, x="y_hat", hue="y", bins=50, stat="probability")
plt.show()

In [ ]:
# 각 클래스별 분포가 양 극단으로 완벽하게 치우쳐져 있다면, 모델이 매우 예측을 잘하고 있다고 판단할 수 있다.